In [13]:
import polars as pl
import matplotlib.pyplot as plt
from event_detection_ap_score import event_detection_ap

import pandas as pd

In [14]:
# BASE_PATH = '/home/matmozaur/Downloads/child-mind-institute-detect-sleep-states/'
BASE_PATH = '../../cmidss/'

In [15]:
x = pl.read_parquet(BASE_PATH+'train_series.parquet')
y = pl.read_csv(BASE_PATH+'train_events.csv')

In [16]:
# x = pd.read_parquet(BASE_PATH+'train_series.parquet')
# y = pd.read_csv(BASE_PATH+'train_events.csv')

In [17]:
s_idxs = list(x['series_id'].unique())

In [18]:
len(s_idxs)

277

# ----------------

In [19]:
y = y.filter(pl.col("series_id").is_in(s_idxs[:100])).to_pandas()

x = x.filter(pl.col("series_id").is_in(s_idxs[:100])).to_pandas()

In [20]:
y = y.dropna()

In [21]:
x['timestamp'] = pd.to_datetime(x['timestamp'].apply(lambda x: x[:19]))
y['timestamp'] = pd.to_datetime(y['timestamp'].apply(lambda x: x[:19]))
x.set_index('timestamp', inplace=True)
y.set_index('timestamp', inplace=True)

x.sort_index(inplace=True)
y.sort_index(inplace=True)

In [ ]:
# r = pd.to_datetime('2017-08-17 09:44:50')
# s= 'ece2561f07e9'

# x[x['series_id']==s]['anglez'][:r].diff().abs().rolling('15min').mean().plot(figsize=(20,8))
# for a, b in zip(y[(y['event']=='onset') & (y['series_id']==s)].index,
#                 y[(y['event']=='wakeup') & (y['series_id']==s)].index):
#     if b < r:
#         plt.axvspan(a, b,alpha=0.2, color='r')

# ----------------------------

In [67]:
def fill_sleep_pred(t, fill='60min', cut='60min'):
    t = t.rolling(fill).max()
    t = t[::-1].rolling(fill).min()[::-1]

    t = t.rolling(cut).min()
    t = t[::-1].rolling(cut).max()[::-1]
    
    t = t.astype(int).diff()[2:]
    t = t[t[t==1].index[0]:]
    return t

def prepare_rows_from_sleep(t2, x, sid):
    rows = []
    last = t2.index[0] - pd.Timedelta('24h')
    i = 0 
    fail_counter = 0
    for a in t2[t2==1].index:
        if a.hour in list(range(18, 25)) + list(range(0, 3)) and a > last + pd.Timedelta('10h'):
            try:
                b = t2[a+pd.Timedelta('3h'):a+pd.Timedelta('14h')][t2[a+pd.Timedelta('3h'):a+pd.Timedelta('14h')]==-1].index[0]
                last = a
                d1 = x.loc[a][['step']].to_dict()
                if type(d1['step']) == dict:
                    d1['step'] = list(d1['step'].values())[0]
                i += 1
                d1['series_id'] = sid
                d1['event'] = 'onset'
                d1['score'] = 1
                rows.append(d1)
                d2 = x.loc[b][['step']].to_dict()
                if type(d2['step']) == dict:
                    d2['step'] = list(d2['step'].values())[0]
                i += 1
                d2['series_id'] = sid
                d2['event'] = 'wakeup'
                d2['score'] = 1
                rows.append(d2)
            except:
#                 b = t2[a:a+pd.Timedelta('12h')].index[-1]
#                 last = a
#                 d1 = x.loc[a][['step']].to_dict()
#                 i += 1
#                 d1['series_id'] = sid
#                 d1['event'] = 'onset'
#                 d1['score'] = 1
#                 rows.append(d1)
#                 d2 = x.loc[b][['step']].to_dict()
#                 if type(d2['step']) == dict:
#                     d2['step'] = list(d2['step'].values())[0]
#                 i += 1
#                 d2['series_id'] = sid
#                 d2['event'] = 'wakeup'
#                 d2['score'] = 0.1
#                 rows.append(d2)
                fail_counter += 1
                continue
    print(f'Failed nights: {fail_counter}')
    return rows

def prepare_sol(x, f, fill='60min', cut='60min'):
    rows = []
    for sid in x['series_id'].unique():
        x2 = x[x['series_id'] == sid]
        t = f(x2)
        t = fill_sleep_pred(t, fill, cut)
        rows += prepare_rows_from_sleep(t, x2, sid)
    pred = pd.DataFrame.from_records(rows)
    pred.sort_values(["series_id","event"], ascending=[0,1], inplace=True)
    pred.reset_index(drop=True, inplace=True)
    pred["row_id"] = pred.index
    pred = pred[["row_id", "series_id","step","event","score"]]
    return pred

def prep_trans(a):
    x_tr = a[['series_id', 'anglez', 'enmo']]

    x_tr['anglez_1_15'] = 0
    x_tr['enmo_1_15'] = 0
    x_tr['anglez_2_5'] = 0
    x_tr['enmo_2_5'] = 0

    for sid in x_tr['series_id'].unique():
        print(sid, end='\r')
        x_tr.loc[x_tr['series_id']==sid,'anglez_1_15'] =\
            x_tr[x_tr['series_id']==sid]['anglez'].diff().abs().rolling('15min').mean()
        x_tr.loc[x_tr['series_id']==sid,'enmo_1_15'] =\
            x_tr[x_tr['series_id']==sid]['enmo'].diff().abs().rolling('15min').mean()
        x_tr.loc[x_tr['series_id']==sid,'anglez_2_5'] =\
            x_tr[x_tr['series_id']==sid]['anglez'].rolling('5min').std()
        x_tr.loc[x_tr['series_id']==sid,'enmo_2_5'] =\
            x_tr[x_tr['series_id']==sid]['enmo'].rolling('5min').std()
    return x_tr

def f1(a, model, tr=0.5):
    x_tr = prep_trans(a)
        
    x_tr.bfill(inplace=True)
    x_tr = x_tr[['anglez_1_15', 'enmo_1_15', 'anglez_2_5', 'enmo_2_5']].values
    pred = model.predict_proba(x_tr)[:,1]
    return (pd.Series(pred, index=a.index) >tr).astype(int)

In [24]:
# pred = prepare_sol(x, lambda a: a['anglez'].diff().abs().rolling('15min').mean() < 1.5)

In [25]:
# event_detection_ap(y, pred)

In [26]:
# len(x)

In [28]:
x_tr = prep_trans(x)

In [29]:
x_tr['sleep'] = 0
for sid in x_tr['series_id'].unique():
    for a, b in zip(y[(y['event']=='onset') & (y['series_id']==sid)].index,
                y[(y['event']=='wakeup') & (y['series_id']==sid)].index):
        x_tr.loc[a: b].loc[(x_tr.loc[a: b]['series_id']==sid),'sleep'] = 1

In [30]:
x_tr.dropna(inplace=True)

y_train =x_tr['sleep'].values

x_train = x_tr[['anglez_1_15', 'enmo_1_15', 'anglez_2_5', 'enmo_2_5']].values

In [32]:
from sklearn.ensemble import RandomForestClassifier

In [60]:
# rf = RandomForestClassifier(n_estimators=10, max_depth=8)

# rf.fit(x_train, y_train)

# rf.score(x_train, y_train)

# # x_tr.bfill(inplace=True)

In [62]:
rf2 = RandomForestClassifier(n_estimators=10, max_depth=10)

In [63]:
split_point = int(len(x_train) * 0.8) 

In [64]:
rf2.fit(x_train[:split_point], y_train[:split_point])

RandomForestClassifier(max_depth=10, n_estimators=10)

In [65]:
rf2.score(x_train[:split_point], y_train[:split_point])

0.8900973443482275

In [66]:
rf2.score(x_train[split_point:], y_train[split_point:])

0.9153926845238345

In [69]:
from functools import partial

In [70]:
pred = prepare_sol(x, partial(f1, model=rf2))

Failed nights: 0
Failed nights: 1
Failed nights: 0
Failed nights: 0
Failed nights: 1
Failed nights: 0
Failed nights: 1
Failed nights: 0
Failed nights: 1
Failed nights: 1
Failed nights: 0
Failed nights: 0
Failed nights: 0
Failed nights: 3
Failed nights: 3
Failed nights: 1
Failed nights: 0
Failed nights: 2
Failed nights: 0
Failed nights: 0
Failed nights: 0
Failed nights: 4
Failed nights: 0
Failed nights: 2
Failed nights: 0
Failed nights: 0
Failed nights: 1
Failed nights: 1
Failed nights: 0
Failed nights: 0
Failed nights: 4
Failed nights: 0
Failed nights: 0
Failed nights: 2
Failed nights: 0
Failed nights: 0
Failed nights: 27
Failed nights: 3
Failed nights: 1
Failed nights: 0
Failed nights: 0
Failed nights: 0
Failed nights: 3
Failed nights: 0
Failed nights: 0
Failed nights: 2
Failed nights: 5
Failed nights: 34
Failed nights: 0
Failed nights: 2
Failed nights: 7
Failed nights: 0
Failed nights: 0
Failed nights: 0
Failed nights: 0
Failed nights: 0
Failed nights: 5
Failed nights: 34
Failed nigh

In [71]:
event_detection_ap(y, pred)

0.2808722702384363